# Dialogue special token scan
- 데이터 구성: dialogue(여러 화자 발화, `\n` 구분)와 summary, 각 대화의 고유 id는 fname. 평가 데이터는 한 대화당 3개 요약이 있을 수 있음.
- 길이 확인: 학습 전 대화/요약 길이(토크나이저 적용 전) 분포를 보고 encoder/decoder max_length를 결정. 토크나이저 후 확인이 더 정확.
- 전처리 예: 자모(ㅋㅋ, ㅇㅇ 등) 존재 여부 확인 후 필요 시 대체(`replace`). 특정 패턴 검출은 `find`/정규표현식 활용.
- 워드클라우드/TF-IDF: 명사 추출(Okt 등) 후 빈도 시각화. 불용어가 많으면 TF-IDF로 중요 단어 확인.
- 개인정보 마스킹: 8종 `#PhoneNumber#`, `#Address#`, `#DateOfBirth#`, `#PassportNumber#`, `#SSN#`, `#CardNumber#`, `#CarNumber#`, `#Email#`; 화자 토큰은 `#PersonN#`.
- 정규표현식: `#\w+#`로 마스킹 토큰 추출, `#\w+\d#`로 화자 토큰 추출해 대화 참여 인원 확인.
- 스페셜 토큰 추가: HF 토크나이저에 `additional_special_tokens`로 마스킹/화자 토큰 등록 후 id 확인.
- 아래 셀에서 train/dev/test CSV의 `#토큰#` 패턴을 집계하고, 관심 토큰이 포함된 모든 행(dialogue, summary)을 출력합니다.
- 경로는 절대경로를 사용합니다.

In [5]:
import pandas as pd
import re
from collections import Counter

data_paths = {
    "train": "/root/NLP_Dialouge_Summarization/data/data/train.csv",
    "dev": "/root/NLP_Dialouge_Summarization/data/data/dev.csv",
    "test": "/root/NLP_Dialouge_Summarization/data/data/test.csv",
}

pattern = re.compile(r"#([^\s#]+)#")
results = {}
for split, path in data_paths.items():
    df = pd.read_csv(path)
    summaries = df["summary"].astype(str) if "summary" in df.columns else []
    texts = df["dialogue"].astype(str).tolist() + list(summaries)
    counter = Counter()
    for text in texts:
        for m in pattern.findall(text):
            counter[m] += 1
    results[split] = counter

for split, counter in results.items():
    print(f"== {split} ==")
    print(f"unique tokens: {len(counter)}")
    for token, cnt in counter.most_common(30):
        print(f"{token}: {cnt}")
    print()


== train ==
unique tokens: 24
Person1: 74776
Person2: 68916
Person3: 454
PhoneNumber: 230
Address: 93
Person4: 41
DateOfBirth: 24
CardNumber: 14
Email: 14
CarNumber: 8
Person6: 8
PassportNumber: 7
Person5: 5
Name: 5
PersonName: 5
Bob: 4
SSN: 3
Alex: 3
Kristin: 3
Person7: 3
Price: 1
Liliana: 1
FlightNumber: 1
Mike: 1

== dev ==
unique tokens: 7
Person1: 2991
Person2: 2753
Person3: 7
PhoneNumber: 5
DateOfBirth: 3
Address: 2
Person4: 1

== test ==
unique tokens: 6
Person1: 2518
Person2: 2306
Person3: 15
PhoneNumber: 5
Address: 1
PassportNumber: 1



## 개인정보성 토큰 포함 샘플 전체 출력
- split별로 지정 토큰이 포함된 dialogue와 gold summary를 모두 출력합니다.

In [6]:
def print_rows_with_non_person_tokens(split: str):
    path = data_paths[split]
    df = pd.read_csv(path)
    def has_non_person_token(text: str) -> bool:
        matches = pattern.findall(str(text))
        non_person = [m for m in matches if not re.fullmatch(r"Person\d+", m)]
        return len(non_person) > 0
    mask = df["dialogue"].apply(has_non_person_token)
    filtered = df[mask].copy()
    print(f"\n==== {split.upper()} ({len(filtered)} rows) ====")
    for i, row in filtered.iterrows():
        dialogue = str(row["dialogue"])
        summary = str(row["summary"]) if "summary" in row else "<no summary>"
        fname = str(row.get("fname", i))
        print(f"[fname={fname}]\nDIALOGUE:\n{dialogue}\nSUMMARY:\n{summary}\n" + "-"*40)

print_rows_with_non_person_tokens("train")
print_rows_with_non_person_tokens("dev")
print_rows_with_non_person_tokens("test")



==== TRAIN (250 rows) ====
[fname=train_42]
DIALOGUE:
#Person1#: 여보세요, 여기는 Smith 의사님 병원인가요?
#Person2#: 네, 맞습니다. 어떻게 도와드릴까요?
#Person1#: 네, Smith 의사님과 통화하고 싶은데요.
#Person2#: Smith 의사님은 오늘 오후에 집에 가셨어요. 실례지만, 누구신지 여쭤봐도 될까요?
#Person1#: Jim White입니다.
#Person2#: 아, 네. White 씨군요. Smith 의사님께서 집 전화번호를 드리라고 하셨어요.
#Person1#: 잠시만요. 네, 번호가 어떻게 되나요?
#Person2#: 번호는 #PhoneNumber#입니다.
#Person1#: 정말 감사합니다.
#Person2#: 괜찮습니다.
SUMMARY:
Jim이 Smith 의사에게 전화를 걸었지만 그가 부재중이라 병원 직원이 그의 전화번호를 제공했다.
----------------------------------------
[fname=train_67]
DIALOGUE:
#Person1#: 저기 봐, Jim. 저 남자가 저기서 넘어졌어.
#Person2#: 우리 가서 괜찮은지 봐야겠다.
#Person1#: 아저씨... 아저씨? 괜찮으세요? 아저씨?
#Person2#: 대답이 없네. 맥박하고 호흡을 확인하는 게 좋겠어.
#Person1#: 어, 안 돼. 호흡이 없고 맥박도 없어. 911에 전화해.
#Person2#: 여보세요? 네. 누군가가 #Address#에서 쓰러졌습니다. 그는 호흡이 없고 맥박도 잡히지 않습니다. 네, 제 친구가 CPR을 하고 있어요. 알겠습니다. 감사합니다. 앰뷸런스를 보내주신대요. 자, 내가 도와줄게.
SUMMARY:
#Person1#와 #Person2#는 한 남자가 넘어져 의식을 잃은 것을 발견하고, #Person1#은 CPR을 수행하며 #Person2#는 911에 전화합니다.
----------------------------------------
